<p style= "font-size:200%; font-weight:bold">ESG Rating Panel Data</p> 

- Author: Yichen Luo
- Date: 2022-03-19

In [199]:
import pandas as pd
import numpy as np
import re
import heapq
import jellyfish
from IPython.core.interactiveshell import InteractiveShell
from pandarallel import pandarallel

InteractiveShell.ast_node_interactivity = 'all'
#pd.set_option('display.max_rows', 110)
pd.set_option('display.max_columns', 100)
pandarallel.initialize(progress_bar=True, nb_workers=6)

RD = '/home/data/'
WD = '/home/team_crypto/2022_esg_peer/03_wdata/'

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 模糊匹配函数

In [200]:
PUNC = ''' |,|\.|;|:|\||\(|\)|&|'|"'''

REMOVE = [
    '', 'inc', 'fund', 'ltd', 'plt',
    'corporation', 'corp', 'group', 'holdings',
    'co', 'sa', 'cv', 'llc', 'de', 'the', 'holding', 'na', 'usa', #sa:匿名
    'bv', 'holdco', 'lp', 'plc', 'etf', 'ab', 'and', 'nv', #bv: dutch or belgian version of private limited company; ab: A benificial; nv:nevada company
    'equity', 'enterprises'
]

REPLACE = {
    'bancorp':'bank',
    'banc':'bank',
    'biotechnologies':'biotech',
    'biotechnology':'biotech',
    'technologies':'tech',
    'technology':'tech'
}

def clean_name(s):
    s = s.lower()
    s = re.sub('[\(\[].*[\)\]]', '', s)
    ws = re.split(PUNC, s)
    ws = [w for w in ws if w not in REMOVE]
    return ' '.join([w if w not in REPLACE else REPLACE[w] for w in ws])

def match_top(s, candidates, n=1, method='lev'):
    heap = [(-np.Inf, '') for _ in range(n)]
    heapq.heapify(heap)
    
    for t in candidates:
        l = len(t)
        if method == 'lev':
            score = jellyfish.levenshtein_distance(s, t) /  max(l, len(s)) - 1
        elif method == 'dam_lev':
            score = jellyfish.damerau_levenshtein_distance(s, t) /  max(l, len(s)) - 1
        elif method == 'jaro':
            score = -jellyfish.jaro_distance(s, t)
        elif method == 'jaro_win':
            score = -jellyfish.jaro_winkler_similarity(s, t)
            
        heapq.heappushpop(heap, (-score, t))
        
    heap.sort(reverse=True)
    return heap

clean_name('Xtrackers J.P. Morgan ESG USD High Yield Corporation')

'xtrackers j p morgan esg usd high yield'

# Computstat

## 读取文件

In [201]:
comp = pd.read_csv('/home/team_crypto/2022_esg_peer/02_rdata/comp_na_daily_all.csv')
comp.shape
comp.sample(3)

comp['cusip'].nunique()

(587111, 14)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,curcd,at,costat,conml
477599,114612,20030930,2003.0,INDL,C,D,STD,0069A,01699W004,ALLBRITTON COMMUNICATIONS CO,USD,262.010,A,Allbritton Communications Co
473320,110807,20070430,2006.0,INDL,C,D,STD,HIX,95766J102,WESTERN ASST HIGH INCM FD II,USD,NaN,A,Western Asset High Income Fund II Inc
77027,4555,19881231,1988.0,INDL,C,D,STD,8824B,305915100,FALCONBRIDGE LTD -OLD,CAD,2607.753,I,Falconbridge Ltd -Old


41593

## 提取CUSIP

In [202]:
iscu = comp.drop_duplicates(subset=['cusip'], keep='last')
iscu = iscu.loc[:, ['tic', 'cusip', 'conm']]
iscu.shape
iscu.sample(3)

(41594, 3)

,tic,cusip,conm
74250,HHGP.,41399Y942,HARRIS & HARRIS GROUP-OLD
313262,3INVI.,459921102,INTL MOVIE GROUP
265859,TEAM.1,878311109,TECHTEAM GLOBAL INC


## 保留ISIN CUSIP共同部分

In [203]:
iscu['iscu'] = [i[0:8] for i in iscu['cusip'].map(str)]

#iscu.loc[iscu['iscu6'].duplicated()].sample()
#iscu.loc[iscu['iscu6'] == '33740F']

iscu6 = iscu.copy()
iscu6['iscu'] = [i[0:6] for i in iscu6['cusip'].map(str)]
iscu6 = iscu6.drop_duplicates(subset=['iscu'], keep='last')

iscu.sample(3)
iscu6.sample(3)

,tic,cusip,conm,iscu
450156,SMV.H,784487308,SMC VENTURES INC,78448730
264664,CADNF,146900105,CASCADES INC,14690010
93276,GMK.B,382766202,GORDON MACKAY STORES - CL B,38276620


,tic,cusip,conm,iscu
556691,PAP.3,69999V935,PACIFIC ASIA PETROLEUM -OLD,69999V
476591,ECLP,278856109,ECLIPSYS CORP,278856
364432,TRUE.,15641R103,CENTRUE FINANCIAL CORP-OLD,15641R


# Reprisk

> Dai Rui, Liang Hao, Ng Lilian, 2021, Socially Responsible Corporate Customer, Journal of Financial Economics

## 读取文件

In [204]:
cid = pd.read_parquet(f'{RD}reprisk/pm_company_identifiers.pq')
cid.shape
cid.head(3)

rri = pd.read_parquet(f'{RD}reprisk/pm_rri_data.pq') 
rri.shape
rri.sample(3)

(181582, 9)

,RepRisk_ID,name,headquarter_country,headquarter_country_code,sectors,url,all_ISINs,primary_ISIN,No_reported_risk_exposure
0,10,Acer Inc,Taiwan,TW,Technology Hardware and Equipment,https://www.acer.com,US0044341065 US0044342055 TW0002353000,TW0002353000,False
1,100,Rio Tinto PLC,United Kingdom of Great Britain and Northern I...,GB,Mining,https://www.riotinto.com/,BRRIOTBDR007 ARDEUT112638 GB0007406639 GB00071...,GB0007188757,False
2,1000,Terrane Metals Corp,Canada,CA,Mining,terranemetals.com,CA88103A1167 US88103A3068 CA88103A1084 CA88103...,CA88103A1084,False


(30505452, 11)

,RepRisk_ID,date,current_RRI,RRI_trend,peak_RRI,peak_RRI_date,RepRisk_rating,country_sector_average,environmental_percentage,social_percentage,governance_percentage
7545631,173112,20130831,0,0,0,NaN,AA,22,0%,0%,0%
19847775,376746,20140430,0,0,0,NaN,BB,54,0%,0%,0%
7320416,171848,20190930,0,0,0,NaN,BB,54,0%,0%,0%


## 检查重复

In [205]:
cid.duplicated(subset=['RepRisk_ID']).sum()

0

## 筛选US RRI

In [206]:
df1 = rri.merge(cid, on='RepRisk_ID', how='left', validate='m:1', indicator=True)
df1['_merge'].value_counts()
df1 = df1.loc[df1['_merge'] == 'both']
df1.dropna(subset=['primary_ISIN'], how='any', inplace=True)

df1['country'] = [z[0:2] for z in df1['primary_ISIN']]

df1 = df1.loc[df1['country'] == 'US', ['RepRisk_ID', 'date', 'current_RRI', 'name', 'all_ISINs', 'primary_ISIN', 'headquarter_country_code']]
df1['date'] = pd.to_datetime(df1['date'].map(str))

df1.shape
df1.sample(3)

both          30505452
left_only            0
right_only           0
Name: _merge, dtype: int64

(830760, 7)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code
19618481,37071,2016-06-30,0,Sangamo Therapeutics Inc (formerly known as Sa...,US8006772052 US8006771062,US8006771062,US
4589469,14903,2010-10-31,0,Avnet Inc,US053807AU73 US053807AE32 US053807AF07 US05380...,US0538071038,US
27138921,6624,2009-10-31,0,Electronic Arts Inc (EA),US285512AD11 US285512AC38 US2855121099,US2855121099,US


In [207]:
#[z[0:2] for z in df1['primary_ISIN']]
df1['headquarter_country_code'].nunique()

70

## 提取ISIN共同部分

In [208]:
df1['iscu'] = [i[2:10] for i in df1['primary_ISIN']]

df1.sample(3)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu
16707147,2477,2011-04-30,0,Black & Decker Corp,US0917X2A463 US091797AM26,US0917971006,US,09179710
4288806,144156,2015-07-31,12,UnityPoint Health,US91338CAA36,US91338CAA36,US,91338CAA
29257860,90258,2019-01-31,6,UPC Holding BV,XS1629969830 XS0909771999 XS0909770249 XS05321...,US90320LAC19,NL,90320LAC


## 匹配

In [209]:
df1 = df1.merge(iscu, on='iscu', how='left', validate='m:1', indicator=True)
df1['_merge'].value_counts()

df1.to_parquet(f'{WD}reprisk_compustat.pq')

um1 = df1.loc[df1['_merge'] == 'left_only', 
             ['RepRisk_ID', 'date', 'current_RRI', 'name', 'all_ISINs', 'primary_ISIN', 'headquarter_country_code', 'iscu']
]
df1 = df1.loc[df1['_merge'] == 'both'].drop(columns=['_merge'])

um1.shape
um1.sample(3)

df1.shape
df1.sample(3)

both          592536
left_only     238224
right_only         0
Name: _merge, dtype: int64

(238224, 8)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu
263462,178766,2010-03-31,0,Rimini Street Inc,US76674Q1159 US76674Q2066 US76674Q3056 US76674...,US76674Q2066,US,76674Q20
149258,131892,2013-03-31,0,Tops Holding II Corp,US89078XAA54 US89078XAB38,US89078XAA54,US,89078XAA
274899,183490,2011-04-30,0,Metropolitan Edison Co,US591894CD00 US591894BN90 US591894BP49 US59189...,US5918942096,US,59189420


(592536, 11)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
507851,47792,2019-12-31,0,Ideanomics Inc (formerly Seven Stars Cloud Gro...,US45166V1061,US45166V1061,US,45166V10,IDEX,45166V106,IDEANOMICS INC
210226,153256,2011-11-30,0,Parsley Energy Inc,US7018771029,US7018771029,US,70187710,PE,701877102,PARSLEY ENERGY INC
205540,152475,2013-05-31,0,Sirius XM Holdings Inc,US82968B1035 US82967NAS71,US82968B1035,US,82968B10,SIRI,82968B103,SIRIUS XM HOLDINGS INC


In [210]:
df1['primary_ISIN'].nunique()

3527

## 二次匹配

In [211]:
um1['iscu'] = [i[2:8] for i in um1['primary_ISIN']]

um1.sample(3)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu
767914,89445,2019-11-30,0,Forever Enterprises Inc (formerly Lincoln Heri...,US3462881034,US3462881034,US,346288
542977,54115,2007-02-28,0,Latteno Food Corp (formerly B&D Food Corp),US5184111038,US5184111038,CA,518411
769969,89768,2009-02-28,0,EDP Escelsa (Espirito Santo Centrais Eletricas...,US29665R1059 US29665RAA32 US29665RAC97,US29665R1059,BR,29665R


In [212]:
um1 = um1.merge(iscu6, on='iscu', how='left', validate='m:1', indicator=True)
um1['_merge'].value_counts()
umc1 = um1.loc[um1['_merge'] == 'left_only'].drop(columns=['_merge', 'tic', 'cusip', 'conm'])
um1 = um1.loc[um1['_merge'] == 'both'].drop(columns=['_merge'])

um1.shape
um1.sample(3)
umc1.shape
umc1.sample(3)

left_only     192696
both           45528
right_only         0
Name: _merge, dtype: int64

(45528, 11)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
41074,132126,2013-11-30,0,Jo-Ann Stores Holdings Inc,US47758PAD15 US47758PAE97 US47758PAJ84 US47758...,US47758PAA75,US,47758P,JAS,47758P307,JO-ANN STORES INC
226234,90677,2015-11-30,0,Rochester Gas & Electric Corp,US771367CE70 US771367BP37 US771367BQ10 US77136...,US7713672084,US,771367,EAS1,771367000,ROCHESTER GAS & ELECTRC CORP
71359,1736,2017-08-31,16,Sprint Communications Inc (formerly Sprint Nex...,US65332VAW37 US852061AA81 US852061AD21 US85206...,US8520611000,US,852061,PCS.1,852061506,SPRINT PCS GROUP


(192696, 8)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu
199211,84156,2017-12-31,21,DCP Midstream LLC,US23311RAH93 US26439XAG88 US26439XAF06 US26439...,US23311RAA41,US,23311R
4936,101869,2012-05-31,0,Rain CII Carbon LLC (formerly CII Carbon LLC),US75079QAA85,US75079QAA85,US,75079Q
135006,477984,2015-07-31,0,Amalgamated Bank (formerly Amalgamated Bank of...,US022663AL23 US022663AK40 US022663AJ76 US02266...,US0226631085,US,022663


## 拼接

In [213]:
df1 = pd.concat([df1, um1])
df1.loc[(df1['iscu'] == '035229') & (df1['date'] == '2007-01-31')]
df1.shape
df1.sample(3)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
85512,1928,2007-01-31,16,Anheuser-Busch Co LLC (formerly Anheuser-Busch...,US035229CQ41 US03522AAG58 US03522AAH32 US03522...,US035229BP76,US,035229,BUD.2,035229103,ANHEUSER-BUSCH COS INC


(638064, 11)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
652146,71945,2018-07-31,0,Huazhu Group Ltd (Huazhu Hotels Group; formerl...,KYG465871120 USG21182AA14 US16949NAC39 US44332...,US44332N1063,CN,44332N10,HTHT,44332N106,HUAZHU GROUP LTD
436816,37891,2008-05-31,0,Digi International Inc,US2537981027,US2537981027,US,25379810,DGII,253798102,DIGI INTERNATIONAL INC
208184,152833,2009-09-30,0,Tribune Publishing Co (formerly Tronc Inc),US89609W1071,US89609W1071,US,89609W10,TPCO,89609W107,TRIBUNE PUBLISHING CO


In [214]:
df1.duplicated(subset=['date', 'name']).sum() #检验重复
df1.loc[df1.duplicated(subset=['date', 'name']) == True]
df1.loc[(df1['date'] == '2007-01-31') & (df1['cusip'] == '76117W109')]

168

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
193032,78008,2007-01-31,0,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193033,78008,2007-02-28,0,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193034,78008,2007-03-31,0,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193035,78008,2007-04-30,0,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193036,78008,2007-05-31,0,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
...,...,...,...,...,...,...,...,...,...,...,...
193195,78008,2020-08-31,25,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193196,78008,2020-09-30,24,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193197,78008,2020-10-31,22,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193198,78008,2020-11-30,21,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC


,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
352800,2412738,2007-01-31,0,Resolute Forest Products Inc (Produits Foresti...,US76117W1099,US76117W1099,US,76117W10,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
131208,4402,2007-01-31,0,Resolute FP Canada Inc (Resolu; PF Resolu Cana...,US76117WAA71 US76117WAB54,US76117WAA71,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193032,78008,2007-01-31,0,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC


## 模糊匹配

In [215]:
umc1['namec'] = umc1['name'].apply(clean_name)
umc1.sample(3)
#umc1.loc[umc1['name'] == 'LeasePlan Corp NV']
cands = set(umc1['namec'])

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,namec
53140,1513937,2011-05-31,0,Rivex Technology Corp,US7695341088,US7695341088,US,769534,rivex tech
199859,84375,2015-12-31,0,MUFG Union Bank NA (formerly Union Bank of Cal...,US90520EAK73 US90520EAH45 US90520EAG61 US90520...,US90520EAB74,US,90520E,mufg union bank
234141,96128,2016-10-31,0,Vadda Energy Corp,US91873M1045,US91873M1045,US,91873M,vadda energy


In [216]:
matched = set(df1.loc[df1['cusip'].notna(), 'cusip'])
len(matched)

3769

In [217]:
cp1 = comp.loc[~comp['cusip'].isin(matched), ['tic', 'cusip', 'conm', 'conml']].drop_duplicates()
cp1.shape
cp1.sample(3)

(37842, 4)

,tic,cusip,conm,conml
326193,SMEZ,78463X392,SPDR EURO STOXX SMALL CP ETF,SPDR EURO STOXX Small Cap ETF
238462,SCNI,80603Q105,SCANNER TECHNOLOGIES CORP,Scanner Technologies Corp
580727,VUS.,92206B104,VANGUARD US TOTAL MKT IDX,Vanguard US Total Market Index ETF (CAD-hedged)


In [218]:
cp1['namec'] = cp1['conml'].apply(clean_name)
cp1.sample(10)

,tic,cusip,conm,conml,namec
425775,ENLV.1,293361101,ENLIVEN MARKETING TECH CORP,Enliven Marketing Technologies Corp,enliven marketing tech
99535,HSC..,420128100,HAWKER SIDDELEY CANADA,Hawker Siddeley Canada,hawker siddeley canada
363765,IFUI,456836105,INFU-TECH INC,Infu-Tech Inc,infu-tech
15535,ACHV.1,039575105,ARCHIVE CORP,Archive Corp,archive
524246,GPT,385002308,GRAMERCY PROPERTY TRUST,Gramercy Property Trust,gramercy property trust
428825,CVNGY,16936X201,CHINA CONVERGENT CORP LTD,China Convergent Corporation Ltd,china convergent
328207,HOG.,44056E100,HORIZONS PIPELNS & ENRGY ETF,Horizons Pipelines & Energy Services Index ETF,horizons pipelines energy services index
391809,SLAW,791120108,UROMED CORP,St. Lawrence Energy Corp,st lawrence energy
411551,PRNB,74257L108,PRINCIPIA BIOPHARMA,Principia Biopharma Inc,principia biopharma
311885,GLDFY,380597500,GOLD FIELDS SO AFRICA,Gold Fields of South Africa Ltd,gold fields of south africa


In [219]:
cp1['match'] = cp1['namec'].parallel_apply(lambda x: match_top(x, cands, 3, 'jaro_win'))
cp1.sample(3)

,tic,cusip,conm,conml,namec,match
564449,PSON,71676J102,PETROSONIC ENERGY INC,Petrosonic Energy Inc,petrosonic energy,"[(0.8745098039215686, petrohawk energy), (0.85..."
557166,PNRC,74050H106,PREMIER ENERGY CORP/FL,Premier Energy Corp,premier energy,"[(0.8657142857142858, premier brands), (0.8232..."
246379,DFTS,244635108,DATAWORLD SOLUTIONS INC,Dataworld Solutions Inc,dataworld solutions,"[(0.790664443915218, profile solutions), (0.77..."


In [220]:
cp1['match1'], cp1['match2'], cp1['match3'] = zip(*list(cp1['match'].values))

for i in range(3):
    i += 1
    cp1[f'score{i}'], cp1[f'm_namec{i}'] = zip(*list(cp1[f'match{i}'].values))
    
cp1.loc[cp1['conml'] == 'Drone Aviation Holding Corp -OLD']

,tic,cusip,conm,conml,namec,match,match1,match2,match3,score1,m_namec1,score2,m_namec2,score3,m_namec3
561722,DRNE.1,26299S939,DRONE AVIATION HOLDING -OLD,Drone Aviation Holding Corp -OLD,drone aviation -old,"[(0.7368421052631579, rovi), (0.72823261858349...","(0.7368421052631579, rovi)","(0.7282326185834957, kaiser foundation hospitals)","(0.722132253711201, rki exploration production)",0.736842,rovi,0.728233,kaiser foundation hospitals,0.722132,rki exploration production


In [221]:
cp1 = cp1.loc[cp1['score1'] == 1, ['tic', 'cusip', 'conm', 'm_namec1']]
cp1.rename(columns={'m_namec1':'namec'}, inplace=True)
cp1.shape
cp1.sample(3)

(231, 4)

,tic,cusip,conm,namec
257747,0189A,74799Q003,QVC INC,qvc
475773,SBP.1,802809103,SANTANDER BANCORP,santander bank
165561,C2,173073008,CITIGROUP GLOBAL MKTS HLDGS,citigroup global markets


In [222]:
umc1 = umc1.merge(cp1, on='namec', how='left', validate='m:m', indicator=True)
umc1['_merge'].value_counts()
umc1 = umc1.loc[umc1['_merge'] == 'both'].drop(columns=['_merge', 'namec'])
umc1['cusip'].drop_duplicates().shape[0]
umc1.shape
umc1.sample(3)

left_only     155904
both           40320
right_only         0
Name: _merge, dtype: int64

231

(40320, 11)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
40498,140415,2007-11-30,0,Santander Bank NA,US80280JTL51 US80280JSJ15 US80280JSU69 US80280...,US80280JAA97,US,80280J,SBP.1,802809103,SANTANDER BANCORP
95295,2994,2008-08-31,16,United Airlines Inc,US90932LAH06 US90932LAG23 US90931GAA76 US90932...,US2107953083,US,210795,UAL1,909279002,UNITED AIRLINES INC -OLD
34897,132035,2017-02-28,0,Premier Brands Group Holdings LLC (Nine West H...,US480081AJ70 US480081AG32 US480081AF58 US48008...,US480081AA61,US,480081,BRND,74048L102,PREMIER BRANDS INC/WY


## 拼接

In [223]:
df1 = pd.concat([df1, umc1])
df1.loc[(df1['iscu'] == '651715') & (df1['date'] == '2012-05-31')]
df1.shape
df1.sample(3)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
11888,109134,2012-05-31,0,NewPage Corp,US651715AA20 US651715AB03 US651715AC85 US65171...,US651715AA20,US,651715,2121A,65199A002,NEWPAGE CORP
11889,109134,2012-05-31,0,NewPage Corp,US651715AA20 US651715AB03 US651715AC85 US65171...,US651715AA20,US,651715,0052A,65199V006,NEWPAGE HOLDING CORP


(678384, 11)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
63193,178732,2009-02-28,0,Fidelity Bank,US316041CC47 US316041CU45 US316041CV28 US31604...,US316041AA09,US,316041,FSBI,315831107,FIDELITY BANCORP INC/PA
693477,77452,2018-10-31,16,Hillshire Brands Co; The (formerly Sara Lee Corp),US4325891095 US803111AM56 US803111AQ60 US80311...,US4325891095,US,43258910,HSH,432589109,HILLSHIRE BRANDS CO
592118,6395,2014-03-31,0,ION Geophysical Corp,US462044AJ74 US4620443063 US462044AF52 US46204...,US4620442073,US,46204420,IO,462044207,ION GEOPHYSICAL CORP


## 债券发行重复

In [224]:
df1.shape
df1.loc[df1.duplicated(subset=['cusip', 'date'])].shape
df1.loc[df1.duplicated(subset=['cusip', 'date'])].sample(3)
df1.loc[(df1['cusip'] == '76117W109')&(df1['date'] == '2007-03-31')].sort_values('primary_ISIN', ascending=True)

df1 = df1.sort_values('primary_ISIN', ascending=True).drop_duplicates(subset=['cusip', 'date'], keep='first')
df1.shape
df1.sample(3)

(678384, 11)

(6384, 11)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
131279,4402,2012-12-31,19,Resolute FP Canada Inc (Resolu; PF Resolu Cana...,US76117WAA71 US76117WAB54,US76117WAA71,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
189398,75626,2012-03-31,0,PBF Energy Co LLC,US69318G2057,US69318G2057,US,69318G,PBF,69318G106,PBF ENERGY INC
219109,89854,2010-02-28,0,Connecticut Light & Power Co,US2075978161 US2075973030 US2075978328 US20759...,US2075973030,US,207597,NU1,207597006,CONNECTICUT LIGHT & POWER CO


,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
352802,2412738,2007-03-31,0,Resolute Forest Products Inc (Produits Foresti...,US76117W1099,US76117W1099,US,76117W10,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
131210,4402,2007-03-31,0,Resolute FP Canada Inc (Resolu; PF Resolu Cana...,US76117WAA71 US76117WAB54,US76117WAA71,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC
193034,78008,2007-03-31,0,Resolute Forest Products Inc (Produits Foresti...,US76117WAC38,US76117WAC38,CA,76117W,RFP,76117W109,RESOLUTE FOREST PRODUCTS INC


(672000, 11)

,RepRisk_ID,date,current_RRI,name,all_ISINs,primary_ISIN,headquarter_country_code,iscu,tic,cusip,conm
501526,46275,2010-11-30,0,Santa Cruz County Bank,US8018271069,US8018271069,US,80182710,SCZC,801827106,SANTA CRUZ COUNTY BANK
365468,2652,2012-09-30,14,OM Group Inc,US670872AA86 US670872AB69 US6708721005,US6708721005,US,67087210,OMG,670872100,OM GROUP INC
816366,974358,2011-07-31,0,Akerna Corp,US00973W1100 US00973W2017 US00973W1027,US00973W1027,US,00973W10,KERN,00973W102,AKERNA CORP


## 保存

In [225]:
df1.to_parquet(f'{WD}reprisk_compustat.pq')

# KLD

> Cao Jie, Liang Hao, Zhan Xintong, 2019, Management Science

$$
Raw\ CSR\ Score = \Sigma_{i=1}^{4} (Strengths_{i} - Concerns_{i})
$$
- i denotes four main categories: community, diversity, employee relationship, and environment.

## 读取文件

In [226]:
#nam = pd.read_csv(f'{RD}msci_kld/kldnames.csv')
#nam.shape
#nam.sample(3)

his = pd.read_csv(f'{RD}msci_kld/history.csv', low_memory=False)
his.shape
his.sample(3)
his['year'].describe()

(67500, 169)

,CompanyName,year,issuerid,CUSIP,Ticker,domicile,legacy_companyID,ENV_str_A,ENV_str_B,ENV_str_C,ENV_str_D,ENV_str_F,ENV_str_X,ENV_str_num,ENV_con_A,ENV_con_B,ENV_con_C,ENV_con_D,ENV_con_E,ENV_con_X,ENV_con_num,COM_str_A,COM_str_B,COM_str_C,COM_str_X,COM_str_num,Com_con_A,COM_con_B,COM_con_D,COM_con_X,COM_con_num,HUM_con_A,HUM_con_B,HUM_con_num,EMP_str_A,EMP_str_B,EMP_str_C,EMP_str_D,EMP_str_F,EMP_str_X,EMP_str_num,EMP_con_A,EMP_con_B,EMP_con_C,EMP_con_X,EMP_con_num,DIV_str_A,DIV_str_B,DIV_str_C,DIV_str_D,...,CGOV_str_F,CGOV_con_J,ENV_con_G,ENV_con_H,ENV_con_I,COM_str_H,HUM_con_H,EMP_str_H,EMP_con_F,DIV_str_H,DIV_con_C,PRO_str_D,CGOV_con_K,ENV_str_H,ENV_str_I,ENV_str_J,ENV_con_J,ENV_con_K,HUM_con_J,HUM_con_K,EMP_str_I,EMP_str_J,EMP_str_K,EMP_str_L,EMP_con_G,DIV_con_D,PRO_con_F,CGOV_str_G,CGOV_str_H,GOV_str_num,CGOV_con_L,CGOV_con_M,ENV_str_K,ENV_str_L,ENV_str_M,ENV_str_N,ENV_str_O,ENV_str_P,ENV_str_Q,EMP_str_M,EMP_str_N,EMP_con_H,PRO_str_E,PRO_str_F,PRO_str_G,PRO_str_H,PRO_str_I,PRO_str_J,PRO_str_K,PRO_con_G
19090,Donnelley (R.R.) & Sons Company,2004,NaN,25786710,RRD,NaN,NaN,0,0,0,0,NaN,0,0.0,0.0,0,0.0,0,0.0,1,1.0,0,0,0.0,0,0.0,0,0,0.0,0.0,0.0,NaN,NaN,0.0,0,NaN,0,0,0.0,0,0.0,0,1,0.0,0,1.0,0.0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15700,CommonWealth REIT,2011,NaN,20323310,CWH,NaN,7310.0,0,0,R,0,NaN,0,0.0,NaN,0,NaN,R,NaN,R,0.0,R,R,NaN,0,0.0,NaN,0,NaN,NaN,0.0,NaN,NaN,0.0,R,NaN,R,R,NaN,0,0.0,R,0,NaN,0,0.0,NaN,R,R,R,...,0,R,0,R,R,R,R,0,0,R,1,R,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1121,"ALIGN TECHNOLOGY, INC.",2015,NaN,01625510,ALGN,US,NaN,R,R,R,R,NaN,0,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,R,0,NaN,0,NaN,0,0,NaN,0,NaN,NaN,0,0,NaN,...,NaN,NaN,NaN,0,0,0,NaN,R,0,NaN,0,R,0,R,R,R,0,0,0,0,NaN,NaN,NaN,0,0,NaN,0,0,R,NaN,0,0,R,R,R,R,0,R,R,R,R,0,R,R,R,R,R,R,R,0


count    67500.000000
mean      2009.738785
std          6.787335
min       1991.000000
25%       2005.000000
50%       2011.000000
75%       2015.000000
max       2018.000000
Name: year, dtype: float64

In [227]:
his.loc[(his['CUSIP'] == '89190610') & (his['year'] == 2015)]
his[['CompanyName', 'year', 'issuerid', 'CUSIP', 'Ticker',
    'COM_str_num', 'DIV_str_num', 'EMP_str_num', 'ENV_str_num', 
    'COM_con_num', 'DIV_con_num', 'EMP_con_num', 'ENV_con_num'
   ]].describe()

,CompanyName,year,issuerid,CUSIP,Ticker,domicile,legacy_companyID,ENV_str_A,ENV_str_B,ENV_str_C,ENV_str_D,ENV_str_F,ENV_str_X,ENV_str_num,ENV_con_A,ENV_con_B,ENV_con_C,ENV_con_D,ENV_con_E,ENV_con_X,ENV_con_num,COM_str_A,COM_str_B,COM_str_C,COM_str_X,COM_str_num,Com_con_A,COM_con_B,COM_con_D,COM_con_X,COM_con_num,HUM_con_A,HUM_con_B,HUM_con_num,EMP_str_A,EMP_str_B,EMP_str_C,EMP_str_D,EMP_str_F,EMP_str_X,EMP_str_num,EMP_con_A,EMP_con_B,EMP_con_C,EMP_con_X,EMP_con_num,DIV_str_A,DIV_str_B,DIV_str_C,DIV_str_D,...,CGOV_str_F,CGOV_con_J,ENV_con_G,ENV_con_H,ENV_con_I,COM_str_H,HUM_con_H,EMP_str_H,EMP_con_F,DIV_str_H,DIV_con_C,PRO_str_D,CGOV_con_K,ENV_str_H,ENV_str_I,ENV_str_J,ENV_con_J,ENV_con_K,HUM_con_J,HUM_con_K,EMP_str_I,EMP_str_J,EMP_str_K,EMP_str_L,EMP_con_G,DIV_con_D,PRO_con_F,CGOV_str_G,CGOV_str_H,GOV_str_num,CGOV_con_L,CGOV_con_M,ENV_str_K,ENV_str_L,ENV_str_M,ENV_str_N,ENV_str_O,ENV_str_P,ENV_str_Q,EMP_str_M,EMP_str_N,EMP_con_H,PRO_str_E,PRO_str_F,PRO_str_G,PRO_str_H,PRO_str_I,PRO_str_J,PRO_str_K,PRO_con_G
59010,"TOTAL SYSTEM SERVICES, INC.",2015,NaN,89190610,TSS,US,NaN,R,1,R,0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0,NaN,0,NaN,0,0,NaN,0,NaN,NaN,1,0,NaN,...,NaN,NaN,NaN,0,0,0,NaN,R,0,NaN,0,0,0,0,R,R,0,0,0,0,NaN,NaN,NaN,0,0,NaN,0,R,R,NaN,0,0,R,R,R,R,0,R,R,1,R,0,R,R,R,R,0,R,R,0


,year,COM_str_num,DIV_str_num,EMP_str_num,ENV_str_num,COM_con_num,DIV_con_num,EMP_con_num,ENV_con_num
count,67500.000000,43130.000000,43130.000000,43130.000000,43130.000000,43130.000000,43130.000000,43130.000000,43130.000000
mean,2009.738785,0.173545,0.486691,0.391839,0.247994,0.069882,0.460978,0.327707,0.213123
std,6.787335,0.508136,0.957741,0.889341,0.666858,0.269367,0.638426,0.595690,0.638900
min,1991.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2005.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2011.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2015.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000
max,2018.000000,5.000000,7.000000,9.000000,6.000000,3.000000,3.000000,5.000000,6.000000


## 重新计算KLD Raw Score

### 提取四个组别的子类

In [228]:
df2 = his.loc[:, ['CompanyName', 'year', 'CUSIP', 'Ticker', 'domicile',
                  'COM_str_A', 'COM_str_B', 'COM_str_C', 'COM_str_D', 'COM_str_F', 'COM_str_G', 'COM_str_H', 'COM_str_X',
                  #'COM_con_A', 
                  'COM_con_B', 'COM_con_D', 'COM_con_X',
                  'DIV_str_A', 'DIV_str_B', 'DIV_str_C', 'DIV_str_D', 'DIV_str_E', 'DIV_str_F', 'DIV_str_G', 'DIV_str_H', 'DIV_str_X',
                  'DIV_con_A', 'DIV_con_B', 'DIV_con_C', 'DIV_con_D', 'DIV_con_X',
                  'EMP_str_A', 'EMP_str_B', 'EMP_str_C', 'EMP_str_D', 'EMP_str_F', 'EMP_str_G', 'EMP_str_H', 'EMP_str_M', 'EMP_str_I', 'EMP_str_J', 'EMP_str_K', 'EMP_str_L', 'EMP_str_X', #'EMP_STR_N',
                  'EMP_con_A', 'EMP_con_B', 'EMP_con_C', 'EMP_con_D', 'EMP_con_F', 'EMP_con_G', 'EMP_con_H', 'EMP_con_X', 
                  'ENV_str_A', 'ENV_str_B', 'ENV_str_C', 'ENV_str_D', 'ENV_str_F', 'ENV_str_G', 'ENV_str_H', 'ENV_str_I', 'ENV_str_J', 'ENV_str_X',
                  'ENV_con_A', 'ENV_con_B', 'ENV_con_C', 'ENV_con_D', 'ENV_con_E', 'ENV_con_F', 'ENV_con_G', 'ENV_con_H', 'ENV_con_I', 'ENV_con_J', 'ENV_con_K', 'ENV_con_X'#, 'ENV_STR_K', 'ENV_STR_L', 'ENV_STR_M', 'ENV_STR_N', 'ENV_STR_O', 'ENV_STR_P',
                 ]]

df2.sample(3)

,CompanyName,year,CUSIP,Ticker,domicile,COM_str_A,COM_str_B,COM_str_C,COM_str_D,COM_str_F,COM_str_G,COM_str_H,COM_str_X,COM_con_B,COM_con_D,COM_con_X,DIV_str_A,DIV_str_B,DIV_str_C,DIV_str_D,DIV_str_E,DIV_str_F,DIV_str_G,DIV_str_H,DIV_str_X,DIV_con_A,DIV_con_B,DIV_con_C,DIV_con_D,DIV_con_X,EMP_str_A,EMP_str_B,EMP_str_C,EMP_str_D,EMP_str_F,EMP_str_G,EMP_str_H,EMP_str_M,EMP_str_I,EMP_str_J,EMP_str_K,EMP_str_L,EMP_str_X,EMP_con_A,EMP_con_B,EMP_con_C,EMP_con_D,EMP_con_F,EMP_con_G,EMP_con_H,EMP_con_X,ENV_str_A,ENV_str_B,ENV_str_C,ENV_str_D,ENV_str_F,ENV_str_G,ENV_str_H,ENV_str_I,ENV_str_J,ENV_str_X,ENV_con_A,ENV_con_B,ENV_con_C,ENV_con_D,ENV_con_E,ENV_con_F,ENV_con_G,ENV_con_H,ENV_con_I,ENV_con_J,ENV_con_K,ENV_con_X
62933,"Universal Electronics, Inc.",2008,91348310,UEIC,NaN,0,0,0.0,0.0,0.0,0.0,NaN,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,NaN,0,0,1,NaN,NaN,0.0,0,NaN,0,0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,0.0,NaN,NaN,NaN,1,0,0,0,0,NaN,0,NaN,NaN,NaN,0,0.0,0,0.0,0,0.0,0,NaN,NaN,NaN,NaN,NaN,0
669,"AEON CO.,LTD.",2014,NaN,8267,JP,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,R,NaN,NaN,0,NaN,0,1,NaN,0,0,0,NaN,NaN,NaN,R,R,0,0,NaN,NaN,0,0,0,0,R,0,R,0,NaN,1,0,R,0,R,NaN,R,NaN,0,NaN,0,0,0,0,0,0,0
51432,"Royal Gold, Inc.",2006,78028710,RGLD,NaN,0,0,0.0,0.0,0.0,0.0,NaN,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,NaN,0,0,1,NaN,NaN,0.0,0,NaN,0,0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,0.0,NaN,NaN,NaN,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,0.0,0,0.0,0,0.0,0,NaN,NaN,NaN,NaN,NaN,0


### 将Nan和R转换成0

In [229]:
df2.loc[(df2['CUSIP'] == '15146E10') & (df2['year'] == 2010)]
df2.replace({np.nan:0, 'R':0}, inplace=True)
df2.loc[(df2['CUSIP'] == '15146E10') & (df2['year'] == 2010)]

,CompanyName,year,CUSIP,Ticker,domicile,COM_str_A,COM_str_B,COM_str_C,COM_str_D,COM_str_F,COM_str_G,COM_str_H,COM_str_X,COM_con_B,COM_con_D,COM_con_X,DIV_str_A,DIV_str_B,DIV_str_C,DIV_str_D,DIV_str_E,DIV_str_F,DIV_str_G,DIV_str_H,DIV_str_X,DIV_con_A,DIV_con_B,DIV_con_C,DIV_con_D,DIV_con_X,EMP_str_A,EMP_str_B,EMP_str_C,EMP_str_D,EMP_str_F,EMP_str_G,EMP_str_H,EMP_str_M,EMP_str_I,EMP_str_J,EMP_str_K,EMP_str_L,EMP_str_X,EMP_con_A,EMP_con_B,EMP_con_C,EMP_con_D,EMP_con_F,EMP_con_G,EMP_con_H,EMP_con_X,ENV_str_A,ENV_str_B,ENV_str_C,ENV_str_D,ENV_str_F,ENV_str_G,ENV_str_H,ENV_str_I,ENV_str_J,ENV_str_X,ENV_con_A,ENV_con_B,ENV_con_C,ENV_con_D,ENV_con_E,ENV_con_F,ENV_con_G,ENV_con_H,ENV_con_I,ENV_con_J,ENV_con_K,ENV_con_X
13822,Center Financial Corporation,2010,15146E10,CLFC,NaN,R,R,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,R,R,R,R,NaN,R,R,0,0,1,1,NaN,NaN,R,NaN,R,R,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,R,0,NaN,NaN,0,NaN,NaN,0,0,R,R,0,NaN,0,NaN,NaN,NaN,0,NaN,0,NaN,R,NaN,0,0,R,R,NaN,NaN,R


,CompanyName,year,CUSIP,Ticker,domicile,COM_str_A,COM_str_B,COM_str_C,COM_str_D,COM_str_F,COM_str_G,COM_str_H,COM_str_X,COM_con_B,COM_con_D,COM_con_X,DIV_str_A,DIV_str_B,DIV_str_C,DIV_str_D,DIV_str_E,DIV_str_F,DIV_str_G,DIV_str_H,DIV_str_X,DIV_con_A,DIV_con_B,DIV_con_C,DIV_con_D,DIV_con_X,EMP_str_A,EMP_str_B,EMP_str_C,EMP_str_D,EMP_str_F,EMP_str_G,EMP_str_H,EMP_str_M,EMP_str_I,EMP_str_J,EMP_str_K,EMP_str_L,EMP_str_X,EMP_con_A,EMP_con_B,EMP_con_C,EMP_con_D,EMP_con_F,EMP_con_G,EMP_con_H,EMP_con_X,ENV_str_A,ENV_str_B,ENV_str_C,ENV_str_D,ENV_str_F,ENV_str_G,ENV_str_H,ENV_str_I,ENV_str_J,ENV_str_X,ENV_con_A,ENV_con_B,ENV_con_C,ENV_con_D,ENV_con_E,ENV_con_F,ENV_con_G,ENV_con_H,ENV_con_I,ENV_con_J,ENV_con_K,ENV_con_X
13822,Center Financial Corporation,2010,15146E10,CLFC,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0,0,1,1,0,0.0,0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0,0


### 计算RAW ESG SCORE

In [230]:
df2['esg'] = 0

for i in ['COM_str_A', 'COM_str_B', 'COM_str_C', 'COM_str_D', 'COM_str_F', 'COM_str_G', 'COM_str_H', 'COM_str_X',
          'DIV_str_A', 'DIV_str_B', 'DIV_str_C', 'DIV_str_D', 'DIV_str_E', 'DIV_str_F', 'DIV_str_G', 'DIV_str_H', 'DIV_str_X',
          'EMP_str_A', 'EMP_str_B', 'EMP_str_C', 'EMP_str_D', 'EMP_str_F', 'EMP_str_G', 'EMP_str_H', 'EMP_str_M', 'EMP_str_I', 'EMP_str_J', 'EMP_str_K', 'EMP_str_L', 'EMP_str_X',
          'ENV_str_A', 'ENV_str_B', 'ENV_str_C', 'ENV_str_D', 'ENV_str_F', 'ENV_str_G', 'ENV_str_H', 'ENV_str_I', 'ENV_str_J', 'ENV_str_X'
         ]:
    df2['esg'] = df2['esg'] + df2[i].map(float)

for j in ['COM_con_B', 'COM_con_D', 'COM_con_X',
          'DIV_con_A', 'DIV_con_B', 'DIV_con_C', 'DIV_con_D', 'DIV_con_X',
          'EMP_con_A', 'EMP_con_B', 'EMP_con_C', 'EMP_con_D', 'EMP_con_F', 'EMP_con_G', 'EMP_con_H', 'EMP_con_X', 
          'ENV_con_A', 'ENV_con_B', 'ENV_con_C', 'ENV_con_D', 'ENV_con_E', 'ENV_con_F', 'ENV_con_G', 'ENV_con_H', 'ENV_con_I', 'ENV_con_J', 'ENV_con_K', 'ENV_con_X'
         ]:
    df2['esg'] = df2['esg'] - df2[j].map(float)

df2 = df2.loc[:, ['CompanyName', 'year', 'CUSIP', 'Ticker', 'domicile', 'esg']]
df2.sample(3)

,CompanyName,year,CUSIP,Ticker,domicile,esg
54625,Select Medical Corporation,2003,81619610,SEM,0,1.0
23905,"Fisher Communications, Inc.",2011,33775620,FSCI,0,1.0
28382,Hanover Compressor Company,2006,41076810,HC,0,0.0


## 筛选US KLD

In [231]:
df2 = df2.loc[his['domicile'] == 'US'].copy()
df2.shape
df2.sample(3)

(10121, 6)

,CompanyName,year,CUSIP,Ticker,domicile,esg
41913,NOW INC.,2016,67011P10,DNOW,US,0.0
1448,AMERICAN AIRLINES GROUP INC.,2016,02376R10,AAL,US,4.0
53345,"STARWOOD PROPERTY TRUST, INC.",2015,85571B10,STWD,US,0.0


In [232]:
df2.describe()

,year,esg
count,10121.000000,10121.000000
mean,2016.594803,1.208774
std,1.130833,1.917000
min,2014.000000,-3.000000
25%,2016.000000,0.000000
50%,2017.000000,1.000000
75%,2018.000000,2.000000
max,2018.000000,14.000000


In [233]:
df2.loc[(df2['CUSIP'] == '89190610') & (df2['year'] == 2015)]

,CompanyName,year,CUSIP,Ticker,domicile,esg
59010,"TOTAL SYSTEM SERVICES, INC.",2015,89190610,TSS,US,4.0


## 匹配

In [234]:
iscu.loc[iscu['cusip'] == '88160R101'] #注意到KLD的CUSIP比Computstat里少一位
df2.loc[df2['CUSIP'] == '88160R10'] #并且存在年份重复

,tic,cusip,conm,iscu
570609,TSLA,88160R101,TESLA INC,88160R10


,CompanyName,year,CUSIP,Ticker,domicile,esg
58042,"TESLA MOTORS, INC.",2015,88160R10,TSLA,US,2.0
58043,"TESLA MOTORS, INC.",2016,88160R10,TSLA,US,1.0
58044,"TESLA, INC.",2017,88160R10,TSLA,US,0.0
58045,"TESLA, INC.",2017,88160R10,TSLA,US,0.0
58046,"TESLA, INC.",2018,88160R10,TSLA,US,1.0
58047,"TESLA, INC.",2018,88160R10,TSLA,US,1.0


## 检查cusip(iscu)为0的情况,适合用模糊匹配

In [235]:
df2.loc[(df2['year'] == 2016)&(df2['Ticker'] == 'WTM')]

,CompanyName,year,CUSIP,Ticker,domicile,esg
64819,"WHITE MOUNTAINS INSURANCE GROUP, LTD.",2016,0,WTM,US,0.0


In [236]:
df2.rename(columns={'CUSIP':'iscu'}, inplace=True)
df2 = df2.merge(iscu, on='iscu', how='left', validate='m:1', indicator=True)
df2['_merge'].value_counts()

um2 = df2.loc[df2['_merge'] == 'left_only', 
              ['CompanyName', 'year', 'iscu', 'Ticker', 'domicile', 'esg']            
             ]

df2 = df2.loc[df2['_merge'] == 'both'].drop(columns=['_merge'])

df2.shape
df2.sample(3)

um2.shape
um2.sample(3)

both          9301
left_only      820
right_only       0
Name: _merge, dtype: int64

(9301, 9)

,CompanyName,year,iscu,Ticker,domicile,esg,tic,cusip,conm
403,"AMAZON.COM, INC.",2015,02313510,AMZN,US,-1.0,AMZN,023135106,AMAZON.COM INC
3924,"GRAND CANYON EDUCATION, INC.",2018,38526M10,LOPE,US,1.0,LOPE,38526M106,GRAND CANYON EDUCATION INC
9223,"ULTRATECH, INC.",2016,90403410,UTEK,US,0.0,UTEK,904034105,ULTRATECH INC


(820, 6)

,CompanyName,year,iscu,Ticker,domicile,esg
7146,Parade Technologies Ltd,2018,69900220,4966,US,-1.0
6902,"PIER 1 IMPORTS, INC.",2016,72027910,PIR,US,1.0
8821,THE PROVIDENCE SERVICE CORPORATION,2015,74381510,PRSC,US,1.0


## 二次匹配

In [237]:
um2['iscu'] = [i[0:6] for i in um2['iscu'].map(str)]

um2.sample(3)

,CompanyName,year,iscu,Ticker,domicile,esg
7476,REXNORD CORPORATION,2016,76169B,RXN,US,0.0
1373,BRISTOW GROUP INC.,2016,110394,BRS,US,0.0
4774,ISTAR INC.,2016,45031U,STAR,US,1.0


In [238]:
um2 = um2.merge(iscu6, on='iscu', how='left', validate='m:1', indicator=True)
um2['_merge'].value_counts()
umc2 = um2.loc[um2['_merge'] == 'left_only'].drop(columns=['_merge', 'tic', 'cusip', 'conm'])
um2 = um2.loc[um2['_merge'] == 'both'].drop(columns=['_merge'])
um2.shape
um2.sample(3)
umc2.shape
umc2.sample(3)

left_only     542
both          278
right_only      0
Name: _merge, dtype: int64

(278, 9)

,CompanyName,year,iscu,Ticker,domicile,esg,tic,cusip,conm
737,"Teligent, Inc",2016,87960W,TLGT,US,0.0,TLGTQ,87960W203,TELIGENT INC
70,"ASCENA RETAIL GROUP, INC.",2015,04351G,ASNA,US,2.0,ASNAQ,04351G200,ASCENA RETAIL GROUP INC
459,LIBERTY INTERACTIVE CORPORATION,2015,53071M,QVCA,US,0.0,LMDIA,53071M500,LIBERTY MEDIA ENTERTAINMENT


(542, 6)

,CompanyName,year,iscu,Ticker,domicile,esg
656,"SONUS NETWORKS, INC.",2015,835916,SONS,US,0.0
295,"FOREST CITY ENTERPRISES, INC.",2015,345550,FCE.A,US,1.0
349,HARRIS CORPORATION,2015,413875,HRS,US,2.0


## 拼接

In [239]:
df2 = pd.concat([df2, um2])
df2.loc[(df2['iscu'] == '26817Q') & (df2['year'] == 2015)]
df2.shape
df2.sample(3)

,CompanyName,year,iscu,Ticker,domicile,esg,tic,cusip,conm
250,"DYNEX CAPITAL, INC.",2015,26817Q,DX,US,1.0,DX,26817Q886,DYNEX CAPITAL INC


(9579, 9)

,CompanyName,year,iscu,Ticker,domicile,esg,tic,cusip,conm
519,AMERICAN VANGUARD CORPORATION,2015,03037110,AVD,US,0.0,AVD,030371108,AMERICAN VANGUARD CORP
6232,"NEWPARK RESOURCES, INC.",2018,65171850,NR,US,1.0,NR,651718504,NEWPARK RESOURCES
5860,"MOLINA HEALTHCARE, INC.",2018,60855R10,MOH,US,0.0,MOH,60855R100,MOLINA HEALTHCARE INC


## 模糊匹配

In [240]:
umc2['namec'] = umc2['CompanyName'].apply(clean_name)
umc2.sample(3)
#umc1.loc[umc1['name'] == 'LeasePlan Corp NV']
cands = set(umc2['namec'])

,CompanyName,year,iscu,Ticker,domicile,esg,namec
220,"Cree, Inc.",2016,225447,CREE,US,4.0,cree
280,FEDERATED NATIONAL HOLDING COMPANY,2015,31422T,FNHC,US,0.0,federated national company
307,FRONTIER COMMUNICATIONS CORPORATION,2016,35906A,FTR,US,2.0,frontier communications


In [241]:
matched = set(df2.loc[df2['cusip'].notna(), 'cusip'])
len(matched)

2805

In [242]:
cp2 = comp.loc[~comp['cusip'].isin(matched), ['tic', 'cusip', 'conm', 'conml']].drop_duplicates()
cp2.shape
cp2.sample(3)

(38806, 4)

,tic,cusip,conm,conml
364832,ANPFF,G03926105,ANPULO FOOD INC,Anpulo Food Inc
459966,PNTU,732763107,PONTOTOC PRODUCTION INC,Pontotoc Production Inc
5508,AGMJF,015644107,ALGOMA CENTRAL CORP,Algoma Central Corp


In [243]:
cp2['namec'] = cp2['conml'].apply(clean_name)
cp2.sample(10)

,tic,cusip,conm,conml,namec
412519,EBIZ,37954Y467,GLOBAL X E-COMMERCE ETF,Global X Funds - Global X E-commerce ETF,global x funds - global x e-commerce
559989,SDC.2,81169Q109,SEACLIFF CONSTRUCTION CORP,Seacliff Construction Corp,seacliff construction
262723,EQXXF,29443T100,EQUINOX RESOURCES LTD,Equinox Resources Ltd,equinox resources
161092,LVCI,52729D300,LEVCOR INTERNATIONAL INC,Levcor International Inc,levcor international
173843,2924B,749706107,RPS PRODUCTS INC,RPS Products Inc,rps products
434453,NTPA,64114K104,NETOPIA INC,Netopia Inc,netopia
416630,MGMT,90470L550,BALLAST SMALL/MID CAP ETF,Ballast Small/Mid Cap ETF,ballast small/mid cap
177473,RLIV,761292101,RETIREMENT LIVING INC,Retirement Living Inc,retirement living
434465,ICTG.1,44929Y101,ICT GROUP INC,ICT Group Inc,ict
133773,MXO,577729205,MAXTOR CORP,Maxtor Corp,maxtor


In [244]:
cp2['match'] = cp2['namec'].parallel_apply(lambda x: match_top(x, cands, 3, 'jaro_win'))
cp2.sample(3)

,tic,cusip,conm,conml,namec,match
382092,PEG.1,713655108,PEREGRINE ENERGY LTD,Peregrine Energy Ltd,peregrine energy,"[(0.6980820105820106, peabody energy), (0.6861..."
86118,FFEX,359360104,FROZEN FOOD EXPRESS INDS,Frozen Food Express Industries Inc,frozen food express industries,"[(0.6794558860920418, fresh del monte produce)..."
293108,BNHB,055918205,BNH BANCSHARES INC,BNH Bancshares Inc,bnh bancshares,"[(0.7941017316017316, union bankshares), (0.64..."


In [245]:
cp2['match1'], cp2['match2'], cp2['match3'] = zip(*list(cp2['match'].values))

for i in range(3):
    i += 1
    cp2[f'score{i}'], cp2[f'm_namec{i}'] = zip(*list(cp2[f'match{i}'].values))
    
cp2.loc[cp2['conml'] == 'Vanguard US Total Market Index ETF (CAD-hedged)']

,tic,cusip,conm,conml,namec,match,match1,match2,match3,score1,m_namec1,score2,m_namec2,score3,m_namec3
580727,VUS.,92206B104,VANGUARD US TOTAL MKT IDX,Vanguard US Total Market Index ETF (CAD-hedged),vanguard us total market index,"[(0.666812865497076, taser international), (0....","(0.666812865497076, taser international)","(0.6617918313570489, sundance energy australia...","(0.6537037037037038, northstar realty finance)",0.666813,taser international,0.661792,sundance energy australia limited,0.653704,northstar realty finance


In [246]:
cp2 = cp2.loc[cp2['score1'] == 1, ['tic', 'cusip', 'conm', 'm_namec1']]
cp2.rename(columns={'m_namec1':'namec'}, inplace=True)
cp2.shape
cp2.sample(3)

(64, 4)

,tic,cusip,conm,namec
344030,FCFS,33768G107,FIRSTCASH HOLDINGS INC,firstcash
333552,AXTA,G0750C108,AXALTA COATING SYSTEMS LTD,axalta coating systems
502837,GNRT,Y26889108,GENER8 MARITIME INC,gener8 maritime


In [247]:
umc2 = umc2.merge(cp2, on='namec', how='left', validate='m:m', indicator=True)
umc2['_merge'].value_counts()
umc2 = umc2.loc[umc2['_merge'] == 'both'].drop(columns=['_merge', 'namec'])
umc2['cusip'].drop_duplicates().shape[0]
umc2.shape
umc2.sample(3)

left_only     416
both          144
right_only      0
Name: _merge, dtype: int64

64

(144, 9)

,CompanyName,year,iscu,Ticker,domicile,esg,tic,cusip,conm
268,INVESCO LTD.,2017,0,IVZ,US,2.0,IVZ,G491BT108,INVESCO LTD
273,INVESCO LTD.,2018,0,IVZ,US,3.0,IVZ.,46130D107,INVESCO INC (CANADA)
224,"Gener8 Maritime, Inc.",2016,0,GNRT,US,-1.0,GNRT,Y26889108,GENER8 MARITIME INC


## 拼接

In [248]:
df2 = pd.concat([df2, umc2])
df2.shape
df2.sample(3)

(9723, 9)

,CompanyName,year,iscu,Ticker,domicile,esg,tic,cusip,conm
5736,"MERITOR, INC.",2016,59001K10,MTOR,US,0.0,MTOR,59001K100,MERITOR INC
728,"ARGAN, INC.",2015,04010E10,AGX,US,0.0,AGX,04010E109,ARGAN INC
408,"AMAZON.COM, INC.",2018,02313510,AMZN,US,-2.0,AMZN,023135106,AMAZON.COM INC


## 去除年份重复

In [249]:
df2.drop_duplicates(subset=['CompanyName', 'year'], inplace=True)
df2.shape
df2.sample(3)

(8445, 9)

,CompanyName,year,iscu,Ticker,domicile,esg,tic,cusip,conm
6132,NCR CORPORATION,2016,62886E10,NCR,US,1.0,NCR,62886E108,NCR CORP
3566,FLUOR CORPORATION,2018,34341210,FLR,US,0.0,FLR,343412102,FLUOR CORP
1883,"CHOICE HOTELS INTERNATIONAL, INC.",2015,16990510,CHH,US,0.0,CHH,169905106,CHOICE HOTELS INTL INC


## 债券发行重复检查

In [250]:
df2.duplicated(subset=['cusip', 'year']).sum()

0

## 保存

In [251]:
df2 = df2.astype(str)
df2.dtypes
df2.sample(3)
df2.to_parquet(f'{WD}kld_compustat.pq')

CompanyName    object
year           object
iscu           object
Ticker         object
domicile       object
esg            object
tic            object
cusip          object
conm           object
dtype: object

,CompanyName,year,iscu,Ticker,domicile,esg,tic,cusip,conm
5244,"LEGGETT & PLATT, INCORPORATED",2015,52466010,LEG,US,1.0,LEG,524660107,LEGGETT & PLATT INC
6622,OWENS CORNING,2017,69074210,OC,US,3.0,OC,690742101,OWENS CORNING
3702,FibroGen Inc,2015,31572Q80,FGEN,US,0.0,FGEN,31572Q808,FIBROGEN INC


# ASSET4

https://wrds-www.wharton.upenn.edu/pages/get-data/thomson-reuters/refinitiv-esg/refinitiv-esg-company-summary-data/

> Dai Rui, Liang Hao, Ng Lilian, 2020, Socially responsible corporate customers, JFE

- CSR score: An equal-weighted CSR rating score (ASSET4)
- Env A score: associated with the environmental pillar of CSR Rating (ASSET4)
- Soc A score: associated with the social responsibility pillar of CSR Rating (ASSET4)
- Product A score: associated with the product responsibility pillar of CSR Rating (ASSET4)

## 读取文件

In [252]:
as4 = pd.read_csv(f'{RD}refinitiv/esg/wrds_company_smry.csv')
as4 = as4.loc[:, ['OrgID', 'FisYear', 'overall_score', 'econ_score', 'envrn_score', 'social_score', 'corpgov_score']]
as4.shape
as4.sample(5)

(97349, 7)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score
2215,11607,2008,0.889672,0.713950,0.875315,0.901776,0.648167
82578,108813627,2020,0.852165,0.763060,0.356070,0.857766,0.928306
29486,33156,2003,0.184536,0.076919,0.387779,0.149034,0.586208
25678,28668,2006,0.196435,0.212065,0.154593,0.069906,0.761099
27985,31506,2016,0.942872,0.901486,0.895386,0.901885,0.901827


In [253]:
cat = pd.read_csv(f'{RD}refinitiv/esg/wrds_esg_catscores.csv')
cat = cat.loc[:, ['OrgID', 'FisYear', 'Score', 'CScore', 
                  'ControversiesScore', 'ResourceUseScore', 'EmissionsScore', 'InnovationScore', 'WorkforceScore',
                  'HumanRightsScore', 'CommunityScore', 'ProductRespScore', 'ManagementScore', 'ShareholdersScore',
                  'CSRStrategyScore'
                 ]]
cat.shape
cat.sample(3)

(97353, 15)

,OrgID,FisYear,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore
31125,40294,2014,0.532715,0.532715,1.000000,0.436709,0.720779,0.457447,0.802239,0.051282,0.488636,0.981481,0.411504,0.141593,0.987013
58620,146650,2016,0.328008,0.328008,0.492958,0.000000,0.000000,0.000000,0.171708,0.000000,0.681495,0.310484,0.630463,0.480934,0.000000
68945,100923717,2017,0.589787,0.589787,1.000000,0.912500,0.736111,0.441176,0.817460,0.871429,0.690476,0.209091,0.595287,0.397145,0.795989


- ASSET4 ID, Cusip, and Isin

In [254]:
info = pd.read_csv(f'{RD}refinitiv/esg/wrds_company_info.csv')
info = info.loc[:,['OrgID', 'Cusip', 'Isin', 'OrgName']]
info.shape
info.sample(3)

(17639, 4)

,OrgID,Cusip,Isin,OrgName
1153,13129,G4715211,GB00BGLP8L22,IMI PLC
14774,112368085,00461U10,US00461U1051,Aclaris Therapeutics Inc
8370,131613,Y2102C10,KR7034020008,Doosan Heavy Industries Constrctn Co Ltd


## 检查duplicate并将CUSIP/ISIN与Scores合并

In [255]:
info.duplicated(subset=['OrgID']).sum()
info.loc[info['OrgID'] == 120013811]

info.dropna(inplace=True)
info.duplicated(subset=['OrgID']).sum()

info.loc[info['OrgID'] == 10357]
info.drop_duplicates(subset=['OrgID'], keep='last', inplace=True)
info.duplicated(subset=['OrgID']).sum()

df3 = as4.merge(info, on=['OrgID'], how='left', validate='m:1', indicator=True)
df3['_merge'].value_counts()

df3 = df3.loc[df3['_merge'] == 'both']
df3.drop(columns=['_merge'], inplace=True)

df3.shape
df3.sample(3)

5692

,OrgID,Cusip,Isin,OrgName
17621,120013811,N9031310,NL0015000IY2,Universal Music Group NV
17622,120013811,NaN,NL0015000IY2,Universal Music Group NV


785

,OrgID,Cusip,Isin,OrgName
67,10357,D0621610,DE000BASF111,BASF SE
68,10357,D0621631,DE000BASF111,BASF SE


0

both          84851
left_only     12498
right_only        0
Name: _merge, dtype: int64

(84851, 10)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName
24327,27249,2016,0.837042,0.883959,0.780679,0.467229,0.857935,57174810,US5717481023,"Marsh & McLennan Companies, Inc."
42670,72105,2005,0.249070,0.053299,0.182233,0.588610,0.538801,G5139415,GB00B646JG43,JJB Sports plc
18098,19889,2020,0.384157,0.885441,0.094003,0.171536,0.610110,04274410,US0427441029,Arrow Financial Corporation


## 检查重复再合并ESG, product score相关

In [256]:
df3.duplicated(subset=['OrgID', 'FisYear']).sum()
cat.duplicated(subset=['OrgID', 'FisYear']).sum()

0

0

In [257]:
df3 = df3.merge(cat, on=['OrgID', 'FisYear'], how='left', validate='1:1', indicator=True)
df3['_merge'].value_counts()

df3 = df3.loc[df3['_merge'] == 'both']
df3.drop(columns=['_merge'], inplace=True)

df3.shape
df3.sample(3)

both          84851
left_only         0
right_only        0
Name: _merge, dtype: int64

(84851, 23)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore
28626,38934,2021,0.925642,0.944150,NaN,0.932934,0.418451,M1586M11,IL0006625771,Bank Hapoalim BM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57719,100111901,2019,0.495315,0.797267,0.742152,0.373988,0.083604,Y3746711,CNE000001FW7,"Hua Xia Bank Co., Limited",0.343312,0.343312,1.0,0.435035,0.471554,0.187697,0.517683,0.0,0.167683,0.0,0.572476,0.021987,0.938343
398,10336,2002,0.450895,0.952204,0.520866,0.350833,0.036429,D0330410,DE0007600801,Altana AG,0.294894,0.294894,1.0,0.000000,0.000000,0.467742,0.391892,0.0,0.689189,0.0,0.743243,0.797297,0.000000


## 筛选US ASSET4

In [258]:
df3['country'] = [z[0:2] for z in df3['Isin']]

df3 = df3.loc[df3['country'] == 'US']
df3.shape
df3.sample(3)

(31943, 24)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country
21750,26249,2018,0.309215,0.379381,0.263713,0.112765,0.786090,47916710,US4791671088,Johnson Outdoors Inc.,0.367658,0.367658,1.000000,0.000000,0.000000,0.5,0.170455,0.0,0.250000,0.243590,0.877752,0.409106,0.0,US
47453,123006,2010,0.425790,0.681870,0.185859,0.143584,0.805054,92343E10,US92343E1029,"Verisign, Inc.",0.236443,0.236443,1.000000,0.075581,0.396341,0.0,0.365942,0.0,0.539855,0.254545,0.160302,0.365327,0.0,US
72082,108387406,2020,0.110408,0.056650,0.098474,0.208890,0.603418,03969710,US0396971071,Ardelyx Inc,0.310205,0.206573,0.102941,0.000000,0.000000,0.0,0.869520,0.0,0.515658,0.293919,0.495362,0.160832,0.0,US


## 匹配

In [259]:
df3['iscu'] = df3['Cusip']
df3 = df3.merge(iscu, on='iscu', how='left', validate='m:1', indicator=True)
df3['_merge'].value_counts()

um3 = df3.loc[df3['_merge'] == 'left_only'].drop(columns=['tic', 'cusip', 'conm', '_merge'])
df3 = df3.loc[df3['_merge'] == 'both'].drop(columns=['_merge'])

um3.shape
um3.sample(3)

df3.shape
df3.sample(3)

both          30959
left_only       984
right_only        0
Name: _merge, dtype: int64

(984, 25)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu
18918,146650,2017,0.300601,0.643558,0.142387,0.124402,0.529058,59523K10,US72346Q1040,Pinnacle Financial Partners Inc,0.334797,0.334797,1.0,0.000000,0.000000,0.0,0.157246,0.000000,0.697101,0.320428,0.548504,0.848813,0.000000,US,59523K10
15267,92292,2009,0.459868,0.299721,0.376809,0.514623,0.711267,00120410,US0012041069,Southern Company Gas,0.467115,0.467115,1.0,0.576923,0.725000,0.0,0.595238,0.000000,0.976190,0.600000,0.504649,0.146178,0.131757,US,00120410
22137,101820435,2019,0.178686,0.039281,0.392484,0.435587,0.385250,G8539710,US86389T1060,Studio City International Holdings Ltd,0.382487,0.382487,1.0,0.174847,0.530899,0.0,0.512876,0.049296,0.858369,0.497674,0.286017,0.349576,0.000000,US,G8539710


(30959, 28)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,tic,cusip,conm
25128,107775320,2021,0.457056,0.785134,0.171793,0.110724,0.851374,90353W10,US90353W1036,Ubiquiti Inc,0.338593,0.185963,0.033333,0.0,0.121429,0.0,0.122549,0.0,0.421569,0.175824,0.871309,0.223829,0.0,US,90353W10,UI,90353W103,UBIQUITI INC
20622,100592104,2017,0.119543,0.051762,0.140831,0.108180,0.465399,09624H20,US09624H2085,BlueLinx Holdings Inc.,0.146283,0.146283,1.000000,0.0,0.150685,0.0,0.345361,0.0,0.025773,0.000000,0.527176,0.118507,0.0,US,09624H20,BXC,09624H208,BLUELINX HOLDINGS INC
2859,21383,2021,0.175461,0.236290,NaN,0.112218,0.727815,45766930,US4576693075,Insmed Incorporated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,45766930,INSM,457669307,INSMED INC


## 二次匹配

In [260]:
um3['iscu'] = [i[0:6] for i in um3['iscu'].map(str)]

um3.sample(3)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu
27285,113142625,2016,0.128786,0.114313,0.270184,0.079103,0.362792,18539C10,US18539C1053,Clearway Energy Inc,0.160169,0.160169,1.0,0.000000,0.017241,0.453125,0.027607,0.000000,0.423313,0.000000,0.328406,0.146744,0.000000,US,18539C
31488,118350015,2020,0.631392,0.079964,0.726343,0.758903,0.733748,69890020,US69047Q1022,Ovintiv Inc,0.584139,0.584139,1.0,0.642857,0.470930,0.000000,0.605455,0.939891,0.543636,0.302041,0.748504,0.467086,0.787125,US,698900
17411,127174,2010,0.484892,0.496813,0.288146,0.346677,0.829596,94105310,US9410531001,"Waste Connections, Inc.",0.251404,0.251404,1.0,0.000000,0.180233,0.353659,0.178261,0.000000,0.662281,0.185393,0.384422,0.300000,0.000000,US,941053


In [261]:
um3 = um3.merge(iscu6, on='iscu', how='left', validate='m:1', indicator=True)
um3['_merge'].value_counts()
umc3 = um3.loc[um3['_merge'] == 'left_only'].drop(columns=['_merge', 'tic', 'cusip', 'conm'])
um3 = um3.loc[um3['_merge'] == 'both'].drop(columns=['_merge'])

um3.shape
um3.sample(3)
umc3.shape
umc3.sample(3)

left_only     777
both          207
right_only      0
Name: _merge, dtype: int64

(207, 28)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,tic,cusip,conm
336,32397,2009,0.185345,0.183666,0.173589,0.130628,0.658570,90921810,US9092181091,Unit Corporation,0.203482,0.203482,1.000000,0.086066,0.000000,0.0,0.215976,0.0,0.500000,0.000000,0.733988,0.242252,0.000000,US,909218,UNTC,909218406,UNIT CORP
613,100073500,2014,0.488607,0.497845,0.413813,0.174636,0.895321,46568510,US4656851056,ITC Holdings Corp.,0.318050,0.318050,0.404762,0.474790,0.233607,0.0,0.075397,0.0,0.738095,0.565000,0.600515,0.086082,0.306839,US,465685,ITC1,465685006,ITC HOLDINGS CORP
782,112462386,2014,0.135428,0.322819,0.105391,0.061733,0.451689,53122987,US5312298707,Liberty Media Formula One,0.119897,0.119897,1.000000,0.000000,0.000000,0.0,0.033019,0.0,0.462264,0.302632,0.030412,0.059278,0.000000,US,531229,FWONK,531229854,LIBERTY MEDIA FORMULA ONE


(777, 25)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu
863,115008862,2014,0.115299,0.104651,0.321754,0.320768,0.099698,83083B10,US83083B1017,Sky Deutschland AG,0.093716,0.093716,1.0,0.054688,0.092308,0.0,0.259434,0.1875,0.084906,0.000000,0.086207,0.063218,0.0,US,83083B
779,112444096,2021,0.048937,0.188813,NaN,0.167252,0.070889,55279C20,US55279C2008,MD Medical Group Investments PLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,55279C
965,119823823,2019,0.080963,0.029298,0.098201,0.141717,0.591587,59833J20,US59833J2069,Midwest Holding Inc,0.134168,0.134168,1.0,0.000000,0.000000,0.0,0.272917,0.0000,0.058333,0.362445,0.172617,0.020880,0.0,US,59833J


## 拼接

In [262]:
df3 = pd.concat([df3, um3])
df3.loc[(df3['iscu'] == '94861A') & (df3['FisYear'] == 2019)]
df3.shape
df3.sample(3)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,tic,cusip,conm
913,117176329,2019,0.062216,0.3374,0.098201,0.070689,0.241609,94861A10,US94861A2078,Weidai Ltd (ADR),0.190098,0.190098,1.0,0.0,0.0,0.0,0.132317,0.0,0.475,0.651178,0.116525,0.311441,0.0,US,94861A,WEI,94861A207,WEIDAI LTD -ADR


(31166, 28)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,tic,cusip,conm
19121,153535,2019,0.671480,0.569892,0.460273,0.407892,0.932328,67802610,US6780261052,"Oil States International, Inc.",0.400102,0.400102,1.0,0.324786,0.324818,0.0,0.375000,0.277174,0.608974,0.221429,0.674052,0.288970,0.515206,US,67802610,OIS,678026105,OIL STATES INTL INC
12314,53108,2003,0.349667,0.581237,0.194085,0.285361,0.582025,07589610,US0758961009,Bed Bath & Beyond Inc.,0.235911,0.235911,1.0,0.000000,0.000000,0.0,0.500000,0.000000,0.580000,0.000000,0.358295,0.252304,0.000000,US,07589610,BBBY,075896100,BED BATH & BEYOND INC
23659,106263312,2009,0.274900,0.382701,0.115859,0.309089,0.621544,58502B10,US58502B1061,MEDNAX Inc,0.304897,0.304897,1.0,0.000000,0.000000,0.0,0.381579,0.000000,0.460526,0.155172,0.587293,0.571798,0.000000,US,58502B10,MD,58502B106,MEDNAX INC


## 模糊匹配

In [263]:
umc3['namec'] = umc3['OrgName'].apply(clean_name)
umc3.sample(3)
cands = set(umc3['namec'])

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,namec
515,111141,2017,0.366330,0.255148,0.491142,0.167788,0.711563,37575310,US91879Q1094,"Vail Resorts, Inc.",0.491335,0.491335,1.000000,0.314159,0.734127,0.000000,0.222527,0.0000,0.659341,0.215152,0.971276,0.888029,0.000000,US,375753,vail resorts
422,66865,2009,0.684527,0.236943,0.926067,0.397800,0.894643,74341088,US7434108889,Prologis,0.483496,0.483496,1.000000,0.500000,0.500000,0.000000,0.500000,0.0000,0.500000,0.500000,0.393079,0.823864,0.914414,US,743410,prologis
702,102745123,2018,0.938690,0.774999,0.947723,0.869626,0.810115,88434410,US8843441028,Thomas Cook Group PLC,0.776774,0.499498,0.222222,0.658088,0.859477,0.648148,0.765700,0.6875,0.867150,0.730159,0.874434,0.826923,0.602469,US,884344,thomas cook


In [264]:
matched = set(df3.loc[df3['cusip'].notna(), 'cusip'])
len(matched)

4103

In [265]:
cp3 = comp.loc[~comp['cusip'].isin(matched), ['tic', 'cusip', 'conm', 'conml']].drop_duplicates()
cp3.shape
cp3.sample(3)

(37508, 4)

,tic,cusip,conm,conml
355976,WGHMF,929267102,WGI HEAVY MINERALS INC,WGI Heavy Minerals Inc
512649,DJJI,25402R101,DIJJI CORP,Dijji Corp
484474,NGHO,64446R207,NEW GENERATION HOLDINGS INC,New Generation Holdings Inc


In [266]:
cp3['namec'] = cp3['conml'].apply(clean_name)
cp3.sample(10)

,tic,cusip,conm,conml,namec
211622,TIC.1,894180108,TRAVELERS CORP,Travelers Corp,travelers
352366,IJIN,459721106,INTERNATIONAL JENSEN INC,International Jensen Inc,international jensen
464116,BRLXF,09950M300,BORALEX INC,Boralex Inc,boralex
211965,TRMW,895883106,TRIANGLE MICROWAVE INC,Triangle Microwave Inc,triangle microwave
540534,DRTX,26658A107,DURATA THERAPEUTICS INC,Durata Therapeutics Inc,durata therapeutics
35448,CAEN,130199102,CALIFORNIA ENGELS MINING CO,California Engels Mining Co,california engels mining
85187,FOX.1,351604103,FOXBORO CO,Foxboro Co,foxboro
417320,LCTU,09290C509,BLACKROCK US CRBN TRNS ETF,Blackrock US Carbon Transition Readiness ETF,blackrock us carbon transition readiness
568004,EUFN,464289180,ISHARES MSCI EUROPE FINL ETF,iShares MSCI Europe Financials ETF,ishares msci europe financials
375755,NHCI,636376105,NATIONAL HOME CENTERS INC,National Home Centers Inc,national home centers


In [267]:
cp3['match'] = cp3['namec'].parallel_apply(lambda x: match_top(x, cands, 3, 'jaro_win'))
cp3.sample(3)

,tic,cusip,conm,conml,namec,match
371248,NPT,6706K4105,NUVEEN PREM INCOME MUN FD 4,Nuveen Premium Income Municipal Fund 4 Inc.,nuveen premium income municipal 4,"[(0.6368686868686869, china national materials..."
306150,GKPRF,36734X104,GATEKEEPER SYSTEMS INC,Gatekeeper Systems Inc,gatekeeper systems,"[(0.6606237816764132, cablevision systems), (0..."
55889,CSMO,221295207,COSMO COMMUNICATIONS CORP,Cosmo Communications Corp,cosmo communications,"[(0.7523809523809524, iheartcommunications), (..."


In [268]:
cp3['match1'], cp3['match2'], cp3['match3'] = zip(*list(cp3['match'].values))

for i in range(3):
    i += 1
    cp3[f'score{i}'], cp3[f'm_namec{i}'] = zip(*list(cp3[f'match{i}'].values))
    
cp3.loc[cp3['conml'] == 'Philex Gold Inc']

,tic,cusip,conm,conml,namec,match,match1,match2,match3,score1,m_namec1,score2,m_namec2,score3,m_namec3
440653,PGI.Z,718141104,PHILEX GOLD INC,Philex Gold Inc,philex gold,"[(0.6265151515151516, prologis), (0.6265151515...","(0.6265151515151516, prologis)","(0.6265151515151516, amphenol)","(0.6161616161616161, dpl)",0.626515,prologis,0.626515,amphenol,0.616162,dpl


In [269]:
cp3 = cp3.loc[cp3['score1'] == 1, ['tic', 'cusip', 'conm', 'm_namec1']]
cp3.rename(columns={'m_namec1':'namec'}, inplace=True)
cp3.shape
cp3.sample(3)

(50, 4)

,tic,cusip,conm,namec
444332,MTN,91879Q109,VAIL RESORTS INC,vail resorts
184426,LQI,50419U202,LA QUINTA CORP,la quinta
406874,BMRC,063425102,BANK OF MARIN BANCORP,bank of marin bank


In [270]:
umc3 = umc3.merge(cp3, on='namec', how='left', validate='m:m', indicator=True)
umc3['_merge'].value_counts()
umc3 = umc3.loc[umc3['_merge'] == 'both'].drop(columns=['_merge', 'namec'])
umc3['cusip'].drop_duplicates().shape[0]
umc3.shape
umc3.sample(3)

left_only     527
both          276
right_only      0
Name: _merge, dtype: int64

50

(276, 28)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,tic,cusip,conm
785,119457827,2021,0.134273,0.464312,NaN,0.083807,0.419112,07331910,US0733191052,BBX Capital Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,073319,BBX,05491N302,BBX CAPITAL CORP
469,133297,2012,0.660591,0.934368,0.412589,0.271704,0.794779,45173410,US4517341073,IHS Inc.,0.256507,0.256507,1.0,0.872222,0.5,0.0,0.097458,0.0,0.283898,0.59901,0.115622,0.148510,0.281136,US,451734,IHS,G4701H109,IHS HOLDING LTD
724,115408462,2006,0.511493,0.524012,0.162022,0.492794,0.827486,85590511,US4414381248,Starwood Hotels & Resorts Worldwide Inc,0.322139,0.322139,1.0,0.000000,0.0,0.0,0.705357,0.0,0.723214,0.00000,0.940691,0.077327,0.000000,US,855905,HOT,85590A401,STARWOOD HOTELS&RESORTS WRLD


## 拼接

In [271]:
df3 = pd.concat([df3, umc3])
df3.shape
df3.sample(3)

(31442, 28)

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,tic,cusip,conm
8959,29999,2019,0.862860,0.397056,0.924979,0.963364,0.616892,75886F10,US75886F1075,Regeneron Pharmaceuticals Inc,0.670989,0.670989,0.911765,0.489362,0.915094,0.000000,0.956349,0.706522,0.993386,0.982558,0.460153,0.270481,0.754919,US,75886F10,REGN,75886F107,REGENERON PHARMACEUTICALS
29943,115599238,2021,0.912067,0.687631,NaN,0.800726,0.852201,24665A10,US24665A1034,Delek US Holdings Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,24665A10,DK,24665A103,DELEK US HOLDINGS INC
28006,114236910,2012,0.209239,0.252411,0.386552,0.415834,0.160398,31354940,US3135494041,Federal-Mogul Holdings Corp,0.371152,0.371152,1.000000,0.259036,0.000000,0.452941,0.308081,0.870370,0.706186,0.354651,0.012847,0.508222,0.098901,US,31354940,FDML,313549404,FEDERAL-MOGUL HOLDINGS CORP


## 债券发行重复检查

In [272]:
df3.loc[df3.duplicated(subset=['cusip', 'FisYear'])]
df3.loc[(df3['cusip'] == '12769G100')&(df3['FisYear'] == 2016)]

,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,tic,cusip,conm
25172,107811723,2015,0.108283,0.085843,0.114694,0.160394,0.381609,49904910,US4990491049,Knight-Swift Transportation Holdings Inc,0.180252,0.180252,1.000000,0.0,0.0,0.0,0.213415,0.000000,0.823171,0.000000,0.139815,0.854012,0.0,US,49904910,KNX,499049104,KNIGHT-SWIFT TRPTN HLDGS INC
25173,107811723,2016,0.280840,0.498424,0.135825,0.298784,0.457915,49904910,US4990491049,Knight-Swift Transportation Holdings Inc,0.275470,0.275470,1.000000,0.0,0.0,0.0,0.313187,0.842105,0.593407,0.171053,0.285990,0.848972,0.0,US,49904910,KNX,499049104,KNIGHT-SWIFT TRPTN HLDGS INC
31702,119167633,2016,0.135642,0.138030,0.127699,0.096165,0.497277,12769G10,US12769G1004,Caesars Entertainment Inc,0.122335,0.122335,1.000000,0.0,0.0,0.0,0.173295,0.000000,0.321023,0.211180,0.044773,0.501928,0.0,US,12769G10,CZR,12769G100,CAESARS ENTERTAINMENT INC
31703,119167633,2017,0.168517,0.164437,0.140383,0.113441,0.571325,12769G10,US12769G1004,Caesars Entertainment Inc,0.188659,0.188659,1.000000,0.0,0.0,0.0,0.255495,0.000000,0.304945,0.536364,0.099243,0.375817,0.0,US,12769G10,CZR,12769G100,CAESARS ENTERTAINMENT INC
31704,119167633,2018,0.273745,0.382011,0.116743,0.109841,0.839096,12769G10,US12769G1004,Caesars Entertainment Inc,0.203873,0.203873,1.000000,0.0,0.0,0.0,0.234300,0.000000,0.338164,0.537037,0.149266,0.442128,0.0,US,12769G10,CZR,12769G100,CAESARS ENTERTAINMENT INC
31705,119167633,2019,0.214884,0.197693,0.104717,0.206065,0.851282,12769G10,US12769G1004,Caesars Entertainment Inc,0.177512,0.177512,0.630952,0.0,0.0,0.0,0.148069,0.000000,0.309013,0.530233,0.068059,0.463022,0.0,US,12769G10,CZR,12769G100,CAESARS ENTERTAINMENT INC
31706,119167633,2020,0.139279,0.101468,NaN,0.151271,0.735592,12769G10,US12769G1004,Caesars Entertainment Inc,0.187797,0.187797,0.663462,0.0,0.0,0.0,0.201852,0.000000,0.351852,0.528226,0.072561,0.476810,0.0,US,12769G10,CZR,12769G100,CAESARS ENTERTAINMENT INC
31728,119195127,2019,0.240195,0.163834,0.168574,0.264735,0.831361,53635D20,US53635D2027,Liquidia Corp,0.410231,0.410231,1.000000,0.0,0.0,0.0,0.855820,0.000000,0.515873,0.286337,0.787536,0.501275,0.0,US,53635D20,LQDA,53635D202,LIQUIDIA CORP
31729,119195127,2020,0.424633,0.458899,0.156366,0.366988,0.868061,53635D20,US53635D2027,Liquidia Corp,0.442420,0.442420,0.676471,0.0,0.0,0.0,0.848643,0.000000,0.901879,0.783784,0.438809,0.668612,0.0,US,53635D20,LQDA,53635D202,LIQUIDIA CORP
918,117265907,2019,0.059302,0.129354,0.103326,0.155121,0.337716,20670411,US2067041168,Brundage-Bone Concrete Pmpg Hldg Inc,0.120649,0.120649,1.000000,0.0,0.0,0.0,0.185294,0.000000,0.458824,0.000000,0.076112,0.889930,0.0,US,206704,BBCP,206704108,CONCRETE PUMPING HOLDINGS


,OrgID,FisYear,overall_score,econ_score,envrn_score,social_score,corpgov_score,Cusip,Isin,OrgName,Score,CScore,ControversiesScore,ResourceUseScore,EmissionsScore,InnovationScore,WorkforceScore,HumanRightsScore,CommunityScore,ProductRespScore,ManagementScore,ShareholdersScore,CSRStrategyScore,country,iscu,tic,cusip,conm
27574,113820716,2016,0.137692,0.13604,0.127893,0.096606,0.509907,12769G10,US12769G1004,Eldorado Resorts Inc,0.112762,0.112762,1.0,0.0,0.0,0.0,0.176301,0.0,0.320809,0.210191,0.046707,0.504305,0.0,US,12769G10,CZR,12769G100,CAESARS ENTERTAINMENT INC
31702,119167633,2016,0.135642,0.13803,0.127699,0.096165,0.497277,12769G10,US12769G1004,Caesars Entertainment Inc,0.122335,0.122335,1.0,0.0,0.0,0.0,0.173295,0.0,0.321023,0.211180,0.044773,0.501928,0.0,US,12769G10,CZR,12769G100,CAESARS ENTERTAINMENT INC


## 保存

In [273]:
df3.to_parquet(f'{WD}asset4_compustat.pq')